In [1]:
from hdx.hdx_configuration import Configuration 
from hdx.data.dataset import Dataset
import requests
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile
import os

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# Population

In [ ]:
#url_pop = 'https://www.regions-et-departements.fr/fichiers/departements-francais.csv'
#download_url(url_pop, 'pop.csv')
#
#live = pd.read_csv('pop.csv', sep='\t')
#live.drop(index=[101,102], inplace=True)
#
#live
#
#live.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
#live['dep_num'] = live['dep_num'].replace({'2A':'201','2B':'202'}).astype(int)
#
#depts = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
#depts.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
#
#
#live == depts

# Covid

In [ ]:
#import requests
#import pandas as pd
#import datetime
#departements = pd.read_csv("data/pop/fr/departements-francais.csv", sep = ";")
#hospi = []
#url = "https://coronavirusapi-france.now.sh/AllLiveData"
#response = requests.get(url).json()
#counter = 0
#for numero in departements.NUMÉRO:
#    nom = str(departements[departements["NUMÉRO"]==numero]["NOM"].values[0])
#    hospi.append((nom, numero, response["allLiveFranceData"][counter]["nouvellesHospitalisations"]))
#    counter+=1
#
#df = pd.DataFrame(hospi, columns =["depname","depnum","newhospi"])
#print(df)
#df.to_csv("covid_daily_data.csv", index = False)

# CAMS

In [ ]:
#to_do (Niclas ?)

# Mobility

In [ ]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)
dataset = Dataset.read_from_hdx('movement-range-maps')
resources = dataset.get_resources()
dic = resources[1]
url_mobility = dic['download_url']

file_mobility = "mvt_range.zip"
download_url(url_mobility, file_mobility)

In [ ]:
with ZipFile(file_mobility, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting mv_range file now...')
    mvt_range = zip.namelist()[-1]
    zip.extract(mvt_range)
    print('Done!')

In [ ]:
with open(mvt_range) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

In [ ]:
data_mob = pd.DataFrame(d[1:], columns=d[0])
data_mob = data_mob[data_mob['country']=='FRA']

In [ ]:
data_mob = data_mob[data_mob['ds']==list(data_mob.iloc[[-1]]['ds'])[0]][['ds', 'polygon_name', 'all_day_bing_tiles_visited_relative_change', 'all_day_ratio_single_tile_users']]

In [ ]:
depts  = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
depts.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
depts = depts.sort_values('dep_num')
depts = depts[:-5]
depts['region'] = depts['region'].replace({'Ile-de-France':'Île-de-France'})
depts

In [ ]:
depts['stay_put'] = 0.0
depts['go_out'] = 0.0

In [ ]:
def add_go_out(row):
    region = row['region']
    go_out = data_mob[data_mob['polygon_name']==region]['all_day_bing_tiles_visited_relative_change']
    return float(list(go_out)[0])
     
def add_stay_put(row):   
    region = row['region']
    stay_put = data_mob[data_mob['polygon_name']==region]['all_day_ratio_single_tile_users']
    return float(list(stay_put)[0])

In [ ]:
depts['go_out'] = depts.apply(add_go_out, axis=1)
depts['stay_put'] = depts.apply(add_stay_put, axis=1)
depts

In [ ]:
def add_stay_put(row)

# Positive test

In [3]:
url_positive_test = 'https://www.data.gouv.fr/es/datasets/r/59aeab47-c364-462c-9087-ce233b6acbbc'
download_url(url_positive_test, 'live_positive_test.csv')

In [5]:
live_test = pd.read_csv('live_positive_test.csv')
live_test

,code_region,region,departement,rang_vaccinal,date_debut_semaine,nb
0,84,ARA,01,1,2021-01-18,1694
1,84,ARA,01,1,2021-01-25,2014
2,84,ARA,01,2,2021-01-25,119
3,84,ARA,01,1,2021-02-01,2017
4,84,ARA,01,2,2021-02-01,1269
...,...,...,...,...,...,...
3361,4,REU,974,2,2021-04-26,3267
3362,4,REU,974,1,2021-05-03,685
3363,4,REU,974,2,2021-05-03,4515
3364,4,REU,974,1,2021-05-10,73
